In [1]:
import numpy as np
import pandas as pd
import zipfile
import dask.dataframe as dk
import tensorflow as tf
import io
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
# temp_dir = "C:/Users/hoang/FileCSV_DACN_2025/ddos_dos_"  # Thư mục lưu file tạm

input_files = [f"file{i+1}.csv" for i in range(3)]

temp_dir =  "2Type_"

input_files = [temp_dir + output_file for output_file in input_files]
print(input_files)
df = [dk.read_csv(file) for file in input_files]

# input_zip = "/mnt/c/Users/hoang/FileCSV_DACN_2025/2Type.zip"
# csv_files = []
# with zipfile.ZipFile(input_zip, 'r') as z:
#     csv_files = [f for f in z.namelist() if f.endswith('.csv')]
# print(csv_files)
# df = [dk.read_csv(f'zip://{file}::{input_zip}') for file in csv_files]
# print(df[0].shape)

2025-04-15 14:23:10.246061: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 14:23:10.428013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744701790.496056     776 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744701790.514509     776 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744701790.653828     776 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

['2Type_file1.csv', '2Type_file2.csv', '2Type_file3.csv']


In [2]:
batch_size = 512
ratio_test_all = 0.15

from dask_ml.model_selection import train_test_split 
# Split 
train_df, test_df = train_test_split(df, test_size=ratio_test_all, random_state=42, shuffle=True)

# def dask_to_tf_dataset(dask_df, batch_size, num_classes): 
#     def generator():
#         for batch in dask_df.to_delayed():
#             batch=batch.compute()  
#             if batch.empty:
#                 continue

#             X = batch.drop(columns='label').values.astype(np.float32)
#             y = batch['label'].values
#             y_onehot = to_categorical(y, num_classes=num_classes)  

#             num_splits = max(1, len(X) // batch_size)  # Đảm bảo không chia nhỏ quá mức
#             X_batches = np.array_split(X, num_splits)
#             y_batches = np.array_split(y_onehot, num_splits)

#             for X_batch, y_batch in zip(X_batches, y_batches):
#                 yield X_batch, y_batch
                
#     output_signature = ( 
#         tf.TensorSpec(shape=(None, 46), dtype=tf.float32), 
#         tf.TensorSpec(shape=(None, 3), dtype=tf.int32),
#     )
    
#     return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)


############## binary ########################
# # load từng batch
def dask_to_tf_dataset(dask_df, batch_size): 
    def generator():
        for batch in dask_df.to_delayed():
            batch = batch.compute()  
            if batch.empty:
                continue

            X = batch.drop(columns='label').values.astype(np.float32)
            y = batch['label'].values.astype(np.int32)  # nhị phân: 0 hoặc 1

            num_splits = max(1, len(X) // batch_size)
            X_batches = np.array_split(X, num_splits)
            y_batches = np.array_split(y, num_splits)

            for X_batch, y_batch in zip(X_batches, y_batches):
                yield X_batch, y_batch

    output_signature = ( 
        tf.TensorSpec(shape=(None, 46), dtype=tf.float32), 
        tf.TensorSpec(shape=(None,), dtype=tf.int32),  # không phải one-hot nữa
    )
    
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)
############ binary ###########

In [3]:
# train_df1, test_df1 = df1.random_split([1 - ratio_test_all, ratio_test_all])
# train_df2, test_df2 = df2.random_split([1 - ratio_test_all, ratio_test_all])
# train_df3, test_df3 = df3.random_split([1 - ratio_test_all, ratio_test_all])
train_dfs = []
test_dfs = []
for dff in df:
    train_df, test_df =dff.random_split([1 - ratio_test_all, ratio_test_all])
    train_dfs.append(train_df)
    test_dfs.append(test_df)
   

train_gens = [dask_to_tf_dataset(train_df, 512).repeat() for train_df in train_dfs]

test_gens = [dask_to_tf_dataset(test_df , 512) for test_df in test_dfs]

I0000 00:00:1744701797.510850     776 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [4]:
import datetime
import numpy as np
import tenseal as ts
#
from server import Server
from client import Client
num_servers = 1
num_clients = 3

stepsPerEpoch_Clients = [int( np.ceil(train_dfs[index].shape[0] / batch_size)) for index in range(num_clients)]

active_servers_list  = ['server_'+str(i)\
                        for i in range(num_servers)]
active_clients_list  = ['client_'+str(i)\
                        for i in range(num_clients)]

print(active_servers_list)
print(active_clients_list)

def init_he_context():
    """Thiết lập context mã hóa đồng hình"""
    context = ts.context(
        ts.SCHEME_TYPE.CKKS, # ckks cho số thực, bfv cho int
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
    context.generate_galois_keys()
    context.global_scale = 2**40
    return context

context = init_he_context()
agents_dict= {}
serverObjects={}
clientObjects={}
serverObjects = {server_name: Server(server_name=server_name, \
                        active_clients_list=active_clients_list) \
                        for server_name in active_servers_list}

clientObjects = {client_name: Client(client_name, train_gens[clientID], test_gens[clientID], \
                        active_clients_list = active_clients_list, he_context=context) \
                        for clientID, client_name in enumerate(active_clients_list)}

for index, client_name in enumerate(active_clients_list):
    clientObjects[client_name].set_steps_per_epoch(stepsPerEpoch_Clients[index])
    clientObjects[client_name].get_steps_per_epoch()
    
# lưu dict
agents_dict['server'] = serverObjects
agents_dict['client'] = clientObjects

# init agents_dict vừa tạo vào client, server
for agent_name, agent in serverObjects.items():
    agent.set_agentsDict(agents_dict=agents_dict)
for agent_name, agent in clientObjects.items():
    agent.set_agentsDict(agents_dict=agents_dict)

client_name = 'client_1'
print("Agent_Dict: ", agents_dict['client'][client_name])

server = agents_dict['server']['server_0']
print(server)

['server_0']
['client_0', 'client_1', 'client_2']
11950
23894
35842
Agent_Dict:  <client.Client object at 0x7f2800bb1970>


In [5]:
if __name__ == '__main__':
    server.InitLoop()
    server.final_statistics()

====================================== Đang chạy Iteration 1 ======================================
Epoch 1/5
Epoch 1/5
Epoch 1/5


I0000 00:00:1744701849.033297     835 service.cc:152] XLA service 0x7f26ec00de00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744701849.033377     835 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-04-15 14:24:09.086846: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744701849.342098     829 cuda_dnn.cc:529] Loaded cuDNN version 90300


   24/23894 ━━━━━━━━━━━━━━━━━━━━ 1:54 5ms/step - accuracy: 0.5575 - loss: 0.7297

2025-04-15 14:24:12.374750: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_transpose_fusion', 224 bytes spill stores, 220 bytes spill loads

I0000 00:00:1744701852.386899     835 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-04-15 14:24:12.389497: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_transpose_fusion', 236 bytes spill stores, 212 bytes spill loads

2025-04-15 14:24:12.461081: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_transpose_fusion', 224 bytes spill stores, 220 bytes spill loads



  173/23894 ━━━━━━━━━━━━━━━━━━━━ 3:56 10ms/step - accuracy: 0.7842 - loss: 0.4825

...

400/400 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - accuracy: 0.9954 - loss: 0.0000e+00
465/600 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.9937 - loss: 0.0000e+00

/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.9910 - loss: 0.0000e+00
Arguments:  Performance Metrics for client_0 on iteration 5 
------------------------------------------- 
local accuracy: 0.9934350848197937 
global accuracy: 0.9906464219093323 
local compute time: 0:07:45.395626 
Simulated time to receive global weights: 0:18:55.457838 
 

600/600 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.9936 - loss: 0.0000e+00
 58/400 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.9934 - loss: 0.0000e+00 

/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


400/400 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9943 - loss: 0.0000e+00
257/600 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.9900 - loss: 0.0000e+00Arguments:  Performance Metrics for client_1 on iteration 5 
------------------------------------------- 
local accuracy: 0.9954740405082703 
global accuracy: 0.9944355487823486 
local compute time: 0:13:01.448095 
Simulated time to receive global weights: 0:18:55.457838 
 

600/600 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.9896 - loss: 0.0000e+00
Arguments:  Performance Metrics for client_2 on iteration 5 
------------------------------------------- 
local accuracy: 0.9935709834098816 
global accuracy: 0.9893463253974915 
local compute time: 0:18:54.605720 
Simulated time to receive global weights: 0:18:55.457838 
 

[client_0] :Simulated time for client set() to finish iteration 5: 0:37:51.385699

[client_1] :Simulated time for client set() to finish iteration 5: 0:37:51.385699

[client_2] :Simulated time for client se

In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.models import load_model

import numpy as np

num_batch_test_eachClient  =  []
for index, test_df in enumerate(test_dfs):
    num_samples_test = test_df.shape[0].compute()
    # Tính số batch
    num_batches_test = int(np.ceil(num_samples_test / batch_size))
    num_batch_test_eachClient.append(num_batches_test)
    
X_tests = {}
Y_tests = {}

for i, client_name in enumerate(active_clients_list):
    X_true_all = []
    y_pred_all = []
    y_true_all = []
    for X_batch, y_batch in test_gens[i].take(num_batch_test_eachClient[i]):
    # .as_numpy_iterator():
        # # take(12000):
        # X_test_list.append(X_batch.numpy())
        # y_test_list.append(y_batch.numpy())  # .numpy()

        # # Gộp tất cả batch lại
        # X_test = np.concatenate(X_test_list, axis=0)
        # y_test = np.concatenate(y_test_list, axis=0)

        # # Nếu y_test đang ở dạng one-hot, chuyển về dạng số
        # y_test = np.argmax(y_test, axis=1)
        
        y_true_all.extend(y_batch.numpy().flatten())
        y_pred_prob = model.predict(X_batch, verbose=0)
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()
        y_pred_all.extend(y_pred)
    
    X_tests[client_name] = X_test
    Y_tests[client_name] = y_test

NameError: name 'model' is not defined

In [ ]:
# tempdirs = ["D:/DoAnChuyenNganh_Train/client_0_log/11h18p__02-04-2025/", "D:/DoAnChuyenNganh_Train/client_1_log/11h18p__02-04-2025/",  "D:/DoAnChuyenNganh_Train/client_2_log/11h18p__02-04-2025/"]
timeFolder="16h53p__13-04-2025/"
tempdirs = [f"DoAnChuyenNganh_Train/client_{i}_log/" for i in range(3)]


model_names =[f"model_{i}.keras" for i in range(5)]
models = {}

for i, client_name in enumerate(active_clients_list):
    models[client_name] = [load_model(tempdirs[i]+model_name) for model_name in model_names]
print (models)

In [ ]:
# Client 0
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

precisions = []
recalls = []
f1s = []

Iterations = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4']
for index in range(len(Iterations)):
    y_pred_prob = models['client_0'][index].predict(X_tests['client_0'])
    # y_pred = y_pred_prob.argmax(axis=1)
    
    # Tính Precision, Recall, F1-score
    precisions.append(precision_score(Y_tests['client_0'], y_pred, average='weighted'))
    recalls.append(recall_score(Y_tests['client_0'], y_pred, average='weighted'))
    f1s.append(f1_score(Y_tests['client_0'], y_pred, average='weighted'))
    
# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 0")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()



cm = confusion_matrix(Y_tests['client_0'], y_pred)

    # Vẽ heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(cm.shape[0]), yticklabels=range(cm.shape[0]))

# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 0")

# Hiển thị
plt.show()


attack_types = ['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
                   'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']
metrics = []

# Số lượng lớp (10 lớp)
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(18, 8), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 0")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client 1
import matplotlib.pyplot as plt

precisions = []
recalls = []
f1s = []
y_pred = []
Iterations = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4']
for index in range(len(Iterations)):
    y_pred_prob = models['client_1'][index].predict(X_tests['client_1'])

    y_pred = y_pred_prob.argmax(axis=1)
    # Tính Precision, Recall, F1-score
    precisions.append(precision_score(Y_tests['client_1'], y_pred, average='weighted'))
    recalls.append(recall_score(Y_tests['client_1'], y_pred, average='weighted'))
    f1s.append(f1_score(Y_tests['client_1'], y_pred, average='weighted'))
    
print("Precision Score: ", precisions)
print("Recall Score: ", recalls)
print("F1 Score: ", f1s)  

# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 1")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()


cm = confusion_matrix(Y_tests['client_1'], y_pred)

# Vẽ heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(cm.shape[0]), yticklabels=range(cm.shape[0]))

# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 1")

# Hiển thị
plt.show()


attack_types =['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
                   'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']
metrics = []

# Số lượng lớp (10 lớp)
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(18, 8), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 1")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client 2
import matplotlib.pyplot as plt

precisions = []
recalls = []
f1s = []
y_pred = []
Iterations = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4']
for index in range(len(Iterations)):
    y_pred_prob = models['client_2'][index].predict(X_tests['client_2'])

    y_pred = y_pred_prob.argmax(axis=1)

    # Tính Precision, Recall, F1-score
    precisions.append(precision_score(Y_tests['client_2'], y_pred, average='weighted'))
    recalls.append(recall_score(Y_tests['client_2'], y_pred, average='weighted'))
    f1s.append(f1_score(Y_tests['client_2'], y_pred, average='weighted'))
    
print("Precision Score: ", precisions)
print("Recall Score: ", recalls)
print("F1 Score: ", f1s)  

# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 2")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()


cm = confusion_matrix(Y_tests['client_2'], y_pred)
# Vẽ heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(cm.shape[0]), yticklabels=range(cm.shape[0]))

# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 2")

# Hiển thị
plt.show()


#
attack_types =['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
                   'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']
metrics = []
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(18, 8), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 2")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client_0 
import matplotlib.pyplot as plt
import numpy as np

file_dir= "client_0_log/11h18p__02-04-2025/"
file_names = ["Iteration_1.csv" ,"Iteration_2.csv", "Iteration_3.csv"]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(12, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

In [ ]:
# Client_1
import matplotlib.pyplot as plt
import numpy as np

file_dir= "client_1_log/11h18p__02-04-2025/"
file_names = ["Iteration_1.csv" ,"Iteration_2.csv", "Iteration_3.csv"]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(12, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

In [ ]:
# Client_2
import matplotlib.pyplot as plt
import numpy as np

file_dir= "client_2_log/11h18p__02-04-2025/"
file_names = ["Iteration_1.csv" ,"Iteration_2.csv", "Iteration_3.csv"]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(12, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs _Client 2")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs Client 2")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()